In [2]:
import warnings
import sys
import os
import pathlib
import torch 
import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import HTML

warnings.filterwarnings('ignore')
sys.path.insert(0, str(pathlib.Path.cwd()/"src"))
%load_ext autoreload
%autoreload 2

import loaders.hrrr
import fields.vector_field

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import loaders.hrrr
import fields.vector_field

In [4]:
date = "2024-09-18"
#"2024-07-19"
level = 500

# Load data
dsf  = loaders.hrrr.discrete_scalar_field(date=date, level=level, hours=4, extent=(-85.5, -75.1, 30.5, 36.5))
dvf  = loaders.hrrr.discrete_vector_field(date=date, level=level, hours=4, extent=(-85.5, -75.1, 30.5, 36.5))



✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-18 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-18 00:00 UTC F01 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-18 00:00 UTC F02 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-18 00:00 UTC F03 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-18 00:00 UTC F04 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-18 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-18 00:00 UTC F01 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-18 00:00 UTC F02 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-18 00:00 UTC F03 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-18 00:00 UTC F04 ┊ GRIB2 @ aws ┊ IDX @ aws


In [5]:
k1 = dsf.coord_field.grid[0]
k2 = dsf.coord_field.grid[1]
XY = dsf.coord_field.XY.reshape(k1, k2, 2)
fac = max(1, 3)
XY = XY[::fac, ::fac, :].reshape(-1,2)
XY.shape[0]*dsf.coord_field.T.size(0)

47430

In [6]:
#fig = dsf.plot(factor = 2, gif = True)
#HTML(fig.to_html5_video())

In [97]:
fig = dvf.plot(factor = 8, gif = True)
HTML(fig.to_html5_video())

In [9]:
dvf.RMS()

9.102

In [199]:
cvf = fields.vector_field.ContinuousVectorField()

In [200]:
cvf.train(dsf, epochs=100, step = 1, factor = 1, size = 5000) 

moved to gpu
Displacement #0 → product of spectral‐norms = 0.2571
Displacement #1 → product of spectral‐norms = 0.2561
Displacement #2 → product of spectral‐norms = 0.2511
Epoch 1/100 — Avg likelihood: 0.5438 — outputscale: 0.6931 — lengthscales: 1.00, 0.44, 0.44 — noise_var: 0.1102
Displacement #0 → product of spectral‐norms = 0.2815
Displacement #1 → product of spectral‐norms = 0.2634
Displacement #2 → product of spectral‐norms = 0.2530
Epoch 2/100 — Avg likelihood: 0.5415 — outputscale: 0.6931 — lengthscales: 1.00, 0.44, 0.44 — noise_var: 0.1206
Displacement #0 → product of spectral‐norms = 0.3410
Displacement #1 → product of spectral‐norms = 0.2901
Displacement #2 → product of spectral‐norms = 0.2611
Epoch 3/100 — Avg likelihood: 0.5550 — outputscale: 0.6931 — lengthscales: 1.00, 0.44, 0.44 — noise_var: 0.1312
Displacement #0 → product of spectral‐norms = 0.4359
Displacement #1 → product of spectral‐norms = 0.3478
Displacement #2 → product of spectral‐norms = 0.2863
Epoch 4/100 — A

In [201]:
fig = cvf.plot(coord_field = dsf.coord_field, factor = 8, gif = True)
HTML(fig.to_html5_video())

In [203]:
[cvf.RMSE(dvf, frame) for frame in range(0, dvf.coord_field.T.size(0))]

[3.56, 3.39, 3.3, 3.42, 3.65]

In [191]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [192]:
cvf.scaled_flow.inspect_weights()

Displacement #0 → product of spectral‐norms = 1.0000
Displacement #1 → product of spectral‐norms = 1.0000
Displacement #2 → product of spectral‐norms = 1.0000
Displacement #3 → product of spectral‐norms = 1.0000
Displacement #4 → product of spectral‐norms = 1.0000


In [253]:
gc.collect()


0

In [89]:
n, m = dsf.coord_field.T.size(0), dsf.coord_field.XY.size(0)          
T = dsf.coord_field.T.repeat_interleave(m).unsqueeze(1)
XY = dsf.coord_field.XY.repeat(n, 1)
Z = dsf.Z.reshape(-1)


In [91]:
T.shape

torch.Size([425475, 1])

In [93]:
XY.shape

torch.Size([425475, 2])

In [32]:
dsf.coord_field.T.shape

torch.Size([5])